# Concatenar o Dataset de Morbidade do SUS num só Dataframe
### http://tabnet.datasus.gov.br/cgi/deftohtm.exe?sih/cnv/nibr.def

In [1]:
# Importe as bibliotecas pandas, numpy, glob e sqlalchemy
import pandas as pd
import numpy as np


In [13]:
# Dica 1: Construa uma lista de arquivos presentes num diretório usando a função glob da biblioteca glob
# Dica 2: Num loop, leia os arquivo duas vezes. Na primeira vez, pra carregar somente a parte desejada do cabeçalho,
#         onde se encontra o mês/ano do respectivo arquivo, utilize o parâmetro nrows. Preste atenção ao separador 
#         dos arquivos CSV, é um ponto e vírgula.
# Dica 3: O método split pode dividir uma string na posição de um caracter especificado.
# Dica 4: Se quiser converter o nome do mês para o seu número, você precisa ajustar a configuração de locale
#         do python para 'pt_BR'. Use o código a seguir para fazer esta operação.
#            import locale
#            locale.setlocale(locale.LC_ALL, 'pt_BR')
# Dica 5: O código strptime(nome_mes,'%b').tm_mon converte nome do mês no seu respectivo número de 1 a 12
# Dica 6: Para criar uma data a partir dos seus atributos use um código semelhante ao exemplo a seguir
#            import datetime
#            datetime.date(year=2020, month=1, day=1)
# Dica 7: Para construir uma nova coluna aplicando uma regra para cada célula use a função apply e construa uma
#         função lambda.
# Dica 8: O método join de uma string consegue unir, com um caracter separador, o conteúdo de uma lista de 
#         elementos numa string. Exemplo: ' '.join(lista_exemplo), produz uma string concatenando com um espaçao
#         os elementos da lista_exemplo. Pode ser útil para isolar o nome do município.
# Dica 9: Adicione todos os dataframes construídos dentro do loop a uma lista e depois use o método concat para 
#         concatenar os dataframes verticalmente. Use o parâmetro ignore_index=True para ignorar o índice dos 
#         dataframes.

def concat_morbidade(files_path):
    """Retorne um dataframe com a concatenação vertical dos dados dos arquivos do diretorio dir_path
    """
    ### BEGIN SOLUTION
    import glob
    from time import strptime
    import datetime
    import locale
    locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
    pieces = []
    result = None
    for file in glob.glob(files_path, recursive=True):
        periodo = pd.read_csv(file, encoding='iso8859-1', skiprows=2, nrows=1)
        mes_ano = periodo.columns.values[0].split(':')[1]
        mes, ano = mes_ano.split('/')
        ano = ano.replace(';','')
        frame = pd.read_csv(file, encoding='iso8859-1', skiprows=3, sep=';', skipfooter=7, engine='python')
        frame['mes'] = mes
        frame['ano'] = ano
        frame['mes_ano'] = datetime.date(year=int(ano), month=strptime(mes,'%b').tm_mon, day=1)
        frame['cod_ibge6'] = frame['Município'].apply(lambda x: x.split(' ')[0]) 
        frame['Município'] = frame['Município'].apply(lambda x: ' '.join(x.split(' ')[1:])) 
        pieces.append(frame)
    # Concatena tudo num único DataFrame
    result = pd.concat(pieces, ignore_index=True)
    
    return result
    ### END SOLUTION


In [14]:
## Faça seus testes aqui
import glob
import pandas as pd
from time import strptime
import datetime
import locale
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
files_path = '../data/originais/morbidade/desagregado/**/*.csv'
lista = glob.glob(files_path)
print(lista)
# df = pd.read_csv(lista[0], encoding='iso8859-1', sep=";", skiprows=2, nrows=1)
# frame = pd.read_csv(lista[0], encoding='iso8859-1', skiprows=3, sep=';', skipfooter=7, engine='python')
# frame['cod_ibge6'] = frame['Município'].str.split(' ')[0]
# frame.head()


['../data/originais/morbidade/desagregado\\2019\\A002344189_28_143_208.csv', '../data/originais/morbidade/desagregado\\2019\\A212356189_28_143_208.csv', '../data/originais/morbidade/desagregado\\2019\\A212407189_28_143_208.csv', '../data/originais/morbidade/desagregado\\2020\\A002126189_28_143_208.csv', '../data/originais/morbidade/desagregado\\2020\\A102654189_28_143_208.csv', '../data/originais/morbidade/desagregado\\2020\\A102744189_28_143_208.csv', '../data/originais/morbidade/desagregado\\2020\\A102812189_28_143_208.csv', '../data/originais/morbidade/desagregado\\2020\\A102850189_28_143_208.csv', '../data/originais/morbidade/desagregado\\2020\\A102927189_28_143_208.csv', '../data/originais/morbidade/desagregado\\2020\\A103139189_28_143_208.csv', '../data/originais/morbidade/desagregado\\2020\\A103238189_28_143_208.csv', '../data/originais/morbidade/desagregado\\2020\\A212148189_28_143_208.csv', '../data/originais/morbidade/desagregado\\2020\\A212323189_28_143_208.csv', '../data/or

In [15]:
# Validação para nota
files_path = '../data/originais/morbidade/desagregado/**/*.csv'
test_data = set(['10899,59', '16793,57', '6585,96', '7446,97', '19054,46', '11462,34'])
morbidade = concat_morbidade(files_path)


In [17]:
morbidade.head()
morbidade.to_csv("../data/originais/morbidade/consolidado/consolidado.csv", index=False)

In [18]:
morbidade.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96711 entries, 0 to 96710
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Município                      96711 non-null  object 
 1   Internações                    96711 non-null  object 
 2   AIH_aprovadas                  96711 non-null  int64  
 3   Valor_total                    96711 non-null  object 
 4   Valor_serviços_hospitalares    96711 non-null  object 
 5   Val_serv_hosp_-_compl_federal  96711 non-null  object 
 6   Val_serv_hosp_-_compl_gestor   96711 non-null  object 
 7   Valor_serviços_profissionais   96711 non-null  object 
 8   Val_serv_prof_-_compl_federal  96711 non-null  object 
 9   Val_serv_prof_-_compl_gestor   96711 non-null  object 
 10  Valor_médio_AIH                96711 non-null  object 
 11  Valor_médio_intern             96711 non-null  object 
 12  Dias_permanência               96711 non-null 